# Skills in OpenAI API

Upload, manage, and attach reusable skills to hosted environments. Agent Skills let you upload and reuse versioned bundles of files in hosted and local shell environments.

## What is a Skill?

A Skill is a reusable bundle of files (instructions + scripts + assets) packaged as a folder and anchored by a required `SKILL.md` manifest. The platform can copy that bundle into an execution environment so the model can read instructions and run code as needed.

In hosted shell, when you attach Skills to the shell tool’s environment (`environment.type="container_auto"`):


- the service uploads and unzips skills into the runtime
- reads `SKILL.md` front matter (name/description) and adds each skill’s `name`, `description`, and `path` to hidden system prompt context so the model knows the skill exists
- if the model decides to invoke a skill, it uses the `path` to read `SKILL.md`, then explores files and executes scripts via the shell tool

Skills are for procedures: repeatable workflows where the how matters (steps, branching logic, formatting rules, scripts) and you want that workflow to be:

- reused across prompts/agents
- versioned and independently shipped
- invoked only when needed (not baked into every system prompt)

### When Skills are particularly appropriate / powerful


**Use Skills when…**

1. **You want something reusable + independently versionable**
Examples: “PowerPoint formatting procedure,” “company-specific report generator,” “standard data-cleaning pipeline.”
2. **The workflow is highly conditional / branched**
If X → do this; else if Y → do that; plus validation + retries.
3. **The workflow needs code execution + local artifacts**
Anything that benefits from scripts, templates, test fixtures, or reference assets that should live beside the instructions. Skills are explicitly designed as “a zip bundle” of those resources.
4. **You want to keep system prompts slim**
Put stable procedures in skills; keep system prompts for global behavior.
5. **Multiple agents/teams should share the same “house style”**
Skills are a nice “org standard library” pattern.
6. **You need reproducibility**
Skills are naturally compatible with version pinning via skill versions (see “Versioning” below).


**Skills are less ideal when…**

- It’s truly a **one-off** task (a quick inline script in the conversation is fine).
- You mostly need **live external data / side effects** (that’s a tool/API call).
- The “procedure” changes every day (skills shine when the workflow stabilizes).


## Skills vs Tools vs System Prompts

People default to heavy system prompts + tool schemas when the boundary isn’t crisp. Here’s a simple framework:

**System prompt = global behavior & constraints**

Use for:
- safety boundaries, tone, refusal style
- “always do X” principles that apply every turn
- small, stable policies

Avoid:

- putting long multi-step procedures here (it bloats every turn and becomes brittle)



**Tools = “do something in the world”**

Use tools when the model must:

- call external services / databases
- create side effects (cancel order, send email)
- fetch live state

**Tools should be:**

- narrowly scoped
- strongly typed inputs
- explicit about side effects

**Skills = packaged procedures (+ code + assets)**

Use a skill when you want the model to:

- follow a repeatable workflow
- use scripts/templates
- execute code in a sandbox
- do it sometimes, not always


## Skill packaging: SKILL.md frontmatter and folder layout

### Folder Structure

A skill is just a folder bundle. Example from the API review doc:
- `SKILL.md` (required)
- optional scripts (`*.py`, `*.js`, …)
- `requirements.txt` and helpers
- assets / templates / sample inputs


### SKILL.md frontmatter

Names and descriptions are expected to come from frontmatter (important for discovery and routing). The guidance is: put name/description in the `SKILL.md` frontmatter. Each API create call uploads one skill bundle (one top-level folder) containing exactly one `SKILL.md`/`skill.md`. To upload multiple skills, upload multiple bundles.


## Creating Skills via API

### Create skill: directory upload or zip upload


The API review doc describes `POST /v1/skills` as `multipart/form-data`, validating “same root folder” and extracting name/description from the manifest frontmatter.

From your slide, both options look like:

**Option A — Upload files (multipart)**



```
curl -X POST 'https://api.openai.com/v1/skills' \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -F 'files[]=@./csv_insights_skill/SKILL.md;filename=csv_insights_skill/SKILL.md;type=text/markdown' \
  -F 'files[]=@./csv_insights_skill/calculate.py;filename=csv_insights_skill/calculate.py;type=text/plain'
```

**Option B — Upload zip**

```
curl -X POST 'https://api.openai.com/v1/skills' \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -F 'files=@./csv_insights_skill.zip;type=application/zip'
```

Practical lesson from internal threads: if you hit server errors, **zipping locally and uploading the zip** was suggested as a workaround.



**Skill object & version pointers**

A skill returns identifiers + version pointers (default/latest). Version pointers show up in platform changes/tests.

## Mounting Skills into execution

### Skills are used via shell + container

To use skills in the Responses API, attach them to the shell tool via `tools[].environment.skills`.

**Ways to reference Skills (hosted + local)**

- Hosted shell: `environment.type="container_auto"`
- Local shell: `environment.type="local"`

Skills can be referenced as:

- `skill_reference` (by `skill_id`, optionally with `version` or `"latest"`)
- `inline` (base64 zip bundle) when you don’t want to create a hosted skill

## Runnable example: `csv_insights_skill` Skill

**1) Create the skill folder**

```
csv_insights_skill/
├── SKILL.md
├── requirements.txt
├── run.py
└── assets/
    └── example.csv
```


**2) `SKILL.md`**

```
---
name: csv-insights
description: Summarize a CSV, compute basic stats, and produce a markdown report + a plot image.
---

# CSV Insights Skill

## When to use this
Use this skill when the user provides a CSV file and wants:
- a quick summary (row/col counts, missing values)
- basic numeric statistics
- a simple visualization
- results packaged into an output folder (or zip)

## Inputs
- A CSV file path (local) or a file mounted in the container.

## Outputs
- `output/report.md`
- `output/plot.png`

## How to run

python -m pip install -r requirements.txt
python run.py --input assets/example.csv --outdir output

```

**3) `run.py`**

In [ ]:
import argparse
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt


def write_report(df: pd.DataFrame, outpath: Path) -> None:
    lines = []
    lines.append(f"# CSV Insights Report\n")
    lines.append(f"**Rows:** {len(df)}  \n**Columns:** {len(df.columns)}\n")
    lines.append("\n## Columns\n")
    lines.append("\n".join([f"- `{c}` ({df[c].dtype})" for c in df.columns]))

    missing = df.isna().sum()
    if missing.any():
        lines.append("\n## Missing values\n")
        for col, count in missing[missing > 0].items():
            lines.append(f"- `{col}`: {int(count)}")
    else:
        lines.append("\n## Missing values\nNo missing values detected.\n")

    numeric = df.select_dtypes(include="number")
    if not numeric.empty:
        lines.append("\n## Numeric summary (describe)\n")
        lines.append(numeric.describe().to_markdown())

    outpath.write_text("\n".join(lines), encoding="utf-8")


def make_plot(df: pd.DataFrame, outpath: Path) -> None:
    numeric = df.select_dtypes(include="number")
    if numeric.empty:
        # No numeric columns → skip plotting
        return

    # Plot the first numeric column as a simple histogram
    col = numeric.columns[0]
    plt.figure()
    df[col].dropna().hist(bins=30)
    plt.title(f"Histogram: {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()


def main() -> None:
    parser = argparse.ArgumentParser()
    parser.add_argument("--input", required=True, help="Path to input CSV")
    parser.add_argument("--outdir", required=True, help="Directory for outputs")
    args = parser.parse_args()

    inpath = Path(args.input)
    outdir = Path(args.outdir)
    outdir.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(inpath)

    write_report(df, outdir / "report.md")
    make_plot(df, outdir / "plot.png")


if __name__ == "__main__":
    main()


**4) Zip it (recommended)**

```
zip -r csv_insights_skill.zip csv_insights_skill
```

**5) Upload the skill**

```
curl -X POST 'https://api.openai.com/v1/skills' \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -F 'files=@./csv_insights_skill.zip;type=application/zip'
```

**6) Run the skill via the API (Hosted shell pattern)**


This follows the flow: create skill → call Responses API with the shell tool and `environment.skills` referencing the skill

Conceptually:


In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
  model="gpt-5.2",
  tools=[{
    "type": "shell",
    "environment": {
      "type": "container_auto",
      "skills": [
        {"type": "skill_reference", "skill_id": "<skill_id>"},
        {"type": "skill_reference", "skill_id": "<skill_id>", "version": 2},
      ],
    },
  }],
  input="Use the skills to analyze the uploaded CSV and write outputs to /mnt/output."
)

print(response.output_text)

**7) Use this Skill via the API (Local container pattern)**

Skills also work with local shell mode. The skill selection and prompt behavior are the same as hosted shell mode, but command execution and filesystem access are still handled by your local runtime.

Conceptually:

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5.2",
    tools=[
        {
            "type": "shell",
            "environment": {
                "type": "local",
                "skills": [
                    {"type": "skill_reference", "skill_id": "<skill_id>"},
                    {"type": "skill_reference", "skill_id": "<skill_id>", "version": 2},
                ],
            },
        }
    ],
    input="Use the configured skills and run locally to summarize today's CSV reports in this repo.",
)

print(response.output_text)

## Operational Best Practices

**1) Keep skills “discoverable”**

* Put a **clear** `name + description` in frontmatter.
* In `SKILL.md`, include: when to use, how to run, expected outputs, gotchas.
  **- Add explicit routing guidance: include “Use when…”, “Don’t use when…”, and a few key edge cases right in `SKILL.md`.**
  **- Include negative examples (when the skill should *not* be triggered) alongside positive examples to improve routing accuracy.**
  **- If routing feels inconsistent, iterate on name/description + examples before changing code.**

This came up directly in “bulk upload” discussions: name/description should come from frontmatter, and you should test with a small number first.

**2) Prefer zip uploads for reliability + reproducibility**

* Zips are portable, easy to version, and a useful workaround when uploads misbehave.

**3) Version pin in production**

You want to be able to say “run this procedure version,” not “whatever the latest is.” Skills are trending toward explicit versions (**default_version**, **latest_version**), and there’s active work on version creation endpoints.

* how to pin (`version: 2`)
* how to float (`version: "latest"`)
* what happens when omitted (defaults to `default_version`)
  **- Consider pinning the model + skill version together for reproducible behavior across deployments.**

**4) Design skills like tiny CLIs**

A good skill script:

* runs from command line
* prints deterministic stdout
* fails loudly with usage/errors
* writes outputs to known file paths when needed
  **- Add concrete templates and worked examples inside the skill (inputs → commands → expected outputs); they cost nothing on turns where the skill isn’t invoked.**
  **- Prefer examples + templates in skills over system-level few-shot prompting when the examples are workflow-specific.**

**5) Avoid duplicating skills in system prompts**

If the system prompt repeats the entire procedure, people will:

* bypass skills
* stuff logic into tool schemas
* and you lose the whole point (reusability + versioning + conditional invocation)

**6) Network Access**

Combining skills + open network access is high-risk; use strict allowlists; treat tool output as untrusted; avoid this configuration for consumer-facing apps where users expect confirmation controls.
**- If network access is required, pair allowlists with explicit “what data is allowed to leave” guidance and treat all tool output as untrusted.**

**7) Use a model that reliably executes multi-step workflows**

**- Skills work best when the model is strong at long-context reasoning and multi-step tool execution (filesystem navigation, CLI runs, verification).**
**- If you see partial completion or brittle execution, upgrade the model or simplify the workflow and add explicit verification steps/output checks in `SKILL.md`.**

**Limits and Validation**

- `SKILL.md` matching is case-insensitive
- Exactly one manifest file allowed (`skill.md`/`SKILL.md`)
- Front matter validation follows Agent Skills spec (name field)
- Max zip upload size: 50 MB
- Max file count per skill version: 500
- Max uncompressed file size: 25 MB

## Conclusion

Skills are the missing “middle layer” between prompts and tools: **prompts** define always-on behavior, **tools** provide atomic capabilities and side effects, and **skills** package repeatable procedures (instructions + scripts + assets) that the model can **mount and execute only when needed.**

A key takeaway from this cookbook: **use skills to keep your system prompts lean and your workflows durable.** Start small—bundle one stable procedure with a clear `SKILL.md`, make it runnable as a tiny CLI, and ship it. Once it’s in production, pin versions for reproducibility, iterate safely by publishing new versions, and treat your skills library like an internal standard library: audited, discoverable, and shared across agents.

As users scale from single-turn assistants to long-running agents, Skills help turn “prompt spaghetti” into **maintainable, testable, versioned workflows**—the kind of building block you can trust, reuse, and evolve over time.